In [ ]:
import sys

sys.path.append("../../../seminars/")

from l5_visualization.scripts.optimizators import DeepResp
import tensorflow as tf
import cv2
from pathlib import Path
import os
import PIL.Image
import numpy as np
import shutil

%load_ext autoreload
%autoreload 2
tf.get_logger().setLevel("INFO")

## Оптимизация вероятности класса для визуализации сети
У нас есть модель, обученная решать задачу классификации: есть на человеке респиратор или нет.  
Модель обучалась на синтетических данных и обобщилась на настоящие тестовые данные.  
Хотим узнать: что смогла выучить модель, чтобы так хорошо обобщиться?


In [ ]:
# load model
resp_class = tf.keras.models.load_model(
    f"../trained_models/resp_class.hdf5", compile=False
)
resp_class.summary()

In [ ]:
# Create the feature extraction model
names = ["dense"]
layers = [resp_class.get_layer(name).output for name in names]

deepdream_model = tf.keras.Model(inputs=resp_class.input, outputs=layers)

In [ ]:
# look at last dense layer's weights
for var in resp_class.trainable_variables:
    if var.name == "dense_1/kernel:0":
        print(f"kernel :{var.numpy()}")
    if var.name == "dense_1/bias:0":
        print(f"bias :{var.numpy()}")

Видим, что активации 0, 4, 5, 6, 7 слоя "dense" умножаются на положительные веса, а активации 1, 2, 3 на отрицательные  
Так так классификатор решает, что на изображении нет респиратора, когда значение акцивации велико (во время обучения этот класс имел метку 1, а другой имел метку 0), то мы можем максимизировать значения этих активаций, чтобы получить изображение, классифицируемое как "лицо без респиратора" 

In [ ]:
def no_resp_max_loss(img, model):
    C = 0.3
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    return (
        layer_activations[0][0] * 1
        + layer_activations[0][4] * 1
        + layer_activations[0][5] * 1
        + layer_activations[0][6] * 1
        + layer_activations[0][7] * 1
        - C * tf.image.total_variation(img)
    )


def generate_samples(loss, out_path, steps=200, lr=0.01, num_samples=5):
    shutil.rmtree(out_path, ignore_errors=True)
    out_path.mkdir(parents=True)
    dr = DeepResp(deepdream_model, loss, steps, lr)
    for i in range(num_samples):
        random_image = tf.random.uniform(
            [64, 64, 3],
            minval=0,
            maxval=None,
            dtype=tf.dtypes.float32,
            seed=None,
            name=None,
        )
        image = dr(random_image)
        image_to_save = cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2BGR)
        image_to_save = cv2.resize(image_to_save, (256, 256))
        cv2.imwrite(str(out_path / f"{i}.jpg"), image_to_save)

In [ ]:
out_root_dir = Path("../outputs/resp_vis_run_1")

In [ ]:
out_path = out_root_dir / Path("no_resp_max")
generate_samples(no_resp_max_loss, out_path, steps=400)

Будем максимизировать вероятность наличия респиратора

In [ ]:
def resp_max_loss(img, model):
    C = 0.3
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    return (
        layer_activations[0][1] * 1
        + layer_activations[0][2] * 1
        + layer_activations[0][3] * 1
        - C * tf.image.total_variation(img)
    )

In [ ]:
out_path = out_root_dir / Path("resp_max")
generate_samples(resp_max_loss, out_path, steps=400)

Начнём с зелёного изображения

In [ ]:
def generate_samples_green_start(loss, out_path, steps=200, lr=0.01, num_samples=5):
    shutil.rmtree(out_path, ignore_errors=True)
    out_path.mkdir(parents=True)
    dr = DeepResp(deepdream_model, loss, steps, lr)
    for i in range(num_samples):
        random_image = tf.random.uniform(
            [64, 64, 3],
            minval=0,
            maxval=None,
            dtype=tf.dtypes.float32,
            seed=None,
            name=None,
        )
        color = np.zeros(
            [64, 64, 3],
            dtype=np.float32,
        )
        color[:, :, 0] = 0.3
        color[:, :, 1] = 0.7
        color[:, :, 2] = 0.3
        color = tf.convert_to_tensor(color) + random_image / 10
        image = dr(color)
        image_to_save = cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2BGR)
        image_to_save = cv2.resize(image_to_save, (256, 256))
        cv2.imwrite(str(out_path / f"{i}.jpg"), image_to_save)

In [ ]:
out_path = out_root_dir / Path("no_resp_max_green")
generate_samples_green_start(no_resp_max_loss, out_path, steps=300)

In [ ]:
out_path = out_root_dir / Path("resp_max_green")
generate_samples_green_start(resp_max_loss, out_path, steps=300)